In [165]:
import pandas as pd
import numpy as np

df = pd.read_csv('../nomad_data/train.csv')
test = pd.read_csv('../nomad_data/test.csv')
print(len(df))
df.head(15)

2400


,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev
0,1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387
1,2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210
2,3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438
3,4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,0.2172,3.3492
4,5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.0505,1.3793
5,6,227,40.0,0.5625,0.4375,0.000,9.8456,5.7176,14.1244,91.1235,91.3033,30.5730,0.2209,2.1633
6,7,206,80.0,0.0312,0.0938,0.875,10.1664,10.1664,10.1666,90.0009,90.0008,89.9987,0.0562,0.9854
7,8,12,20.0,0.5000,0.0000,0.500,12.1350,3.1533,6.2880,89.9997,105.1790,90.0001,0.2243,1.7179
8,9,206,80.0,0.5312,0.2188,0.250,9.4027,9.4023,9.4025,90.0025,90.0030,89.9978,0.2193,2.6257
9,10,194,80.0,0.4062,0.5938,0.000,6.2520,6.2520,23.7931,90.0172,89.9994,119.9997,0.1994,2.5187


In [143]:
X = df[['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]

y1 = df['formation_energy_ev_natom']
y2 = df['bandgap_energy_ev']

In [144]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y1, random_state = 0)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [145]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

linreg = Ridge(alpha=0.0001).fit(X_train_scaled_poly, y_train)

print('(poly deg 2 + ridge) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train_scaled_poly, y_train)))
# print('(poly deg 5 + ridge) R-squared score (test): {:.3f}'
#      .format(linreg.score(X_test_scaled_poly, y_test)))



y_pred1 = linreg.predict(X_train_scaled_poly)
print(sum(y_pred1<0))

y_pred1[y_pred1<0]=0

(poly deg 2 + ridge) R-squared score (training): 0.850
0


In [21]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error


def rmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        if predicted[x]<0 or real.iloc[x]<0:
            print('bad val at', x)
        p = np.log(predicted[x]+1)
        r = np.log(real.iloc[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

rmsle_score = make_scorer(rmsle, greater_is_better=False)
#y_train

#y_pred1[516]
rmsle(y_train, y_pred1)

0.032479542760651542

In [22]:
y_pred1_test = linreg.predict(X_test_scaled_poly)
print(sum(y_pred1_test<0))
y_pred1_test[y_pred1_test<0]=0

rmsle(y_test, y_pred1_test)

0


0.036580880304535081

In [149]:
# X = df[['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
#        'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
#        'lattice_vector_2_ang', 'lattice_vector_3_ang',
#        'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
#        'lattice_angle_gamma_degree']]

X = df[['percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y2, random_state = 0)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [150]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

linreg = Ridge(alpha=0.0001, max_iter=1000).fit(X_train_scaled_poly, y_train)

print('(poly deg 2 + ridge) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train_scaled_poly, y_train)))
# print('(poly deg 5 + ridge) R-squared score (test): {:.3f}'
#      .format(linreg.score(X_test_scaled_poly, y_test)))



y_pred2 = linreg.predict(X_train_scaled_poly)
print(sum(y_pred2<0))

y_pred2[y_pred2<0]=0

y_pred2_ridge = y_pred2

print(rmsle(y_train, y_pred2))

y_pred2_test = linreg.predict(X_test_scaled_poly)
y_pred2_test_ridge = y_pred2_test
print(sum(y_pred2_test<0))
y_pred2_test[y_pred2_test<0]=0

rmsle(y_test, y_pred2_test)

(poly deg 2 + ridge) R-squared score (training): 0.958
0
0.0807597715005
0


0.091165786809164193

In [26]:
rmsle(y_train, y_pred2)

0.079349709878537303

In [99]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

linreg = Lasso(alpha=0.0001, max_iter=10000).fit(X_train_scaled_poly, y_train)

print('(poly deg 3 + lasso) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train_scaled_poly, y_train)))
# print('(poly deg 5 + ridge) R-squared score (test): {:.3f}'
#      .format(linreg.score(X_test_scaled_poly, y_test)))


y_pred2 = linreg.predict(X_train_scaled_poly)
print(sum(y_pred2<0))

y_pred2[y_pred2<0]=0

y_pred2_lasso = y_pred2

rmsle(y_train, y_pred2)

(poly deg 3 + lasso) R-squared score (training): 0.941
2


/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.094099716947186732

In [100]:
poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

from sklearn.linear_model import ElasticNet

elnreg = ElasticNet(alpha=0.001,l1_ratio=0.75,max_iter=5000).fit(X_train_scaled_poly, y_train)

print('(poly deg 3 + elasticnet) R-squared score (training): {:.3f}'
     .format(elnreg.score(X_train_scaled_poly, y_train)))

y_pred2 = elnreg.predict(X_train_scaled_poly)
print(sum(y_pred2<0))

y_pred2[y_pred2<0]=0

rmsle(y_train, y_pred2)

(poly deg 3 + elasticnet) R-squared score (training): 0.878
0


0.12660538327764764

In [114]:
from sklearn.neighbors import KNeighborsRegressor

knnreg = KNeighborsRegressor(n_neighbors = 11, algorithm='auto').fit(X_train_scaled_poly, y_train)

print('R-squared test score: {:.3f}'
     .format(knnreg.score(X_test_scaled_poly, y_test)))

y_pred2 = knnreg.predict(X_train_scaled_poly)
print(sum(y_pred1<0))

y_pred2[y_pred1<0]=0

y_pred2_knn = y_pred2

print(rmsle(y_train, y_pred2))

y_pred2_test = knnreg.predict(X_test_scaled_poly)
print(sum(y_pred2_test<0))
y_pred2_test[y_pred2_test<0]=0

rmsle(y_test, y_pred2_test)

R-squared test score: 0.938
0
0.0977587635079
0


0.10318063534345333

In [131]:
poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

from sklearn.neural_network import MLPRegressor

mlpreg = MLPRegressor(hidden_layer_sizes = [10,10],
                             activation = 'logistic',
                             alpha = 0.003,   #0.0003,
                             solver = 'lbfgs',
                             random_state=101).fit(X_train_scaled_poly, y_train)

print('R-squared test score: {:.3f}'
     .format(mlpreg.score(X_test_scaled_poly, y_test)))

y_pred2 = mlpreg.predict(X_train_scaled_poly)
print(sum(y_pred2<0))

y_pred2[y_pred2<0]=0

y_pred2_mlp = y_pred2

y_pred2_test = mlpreg.predict(X_test_scaled_poly)
y_pred2_test_mlp = y_pred2_test

rmsle(y_train, y_pred2)

R-squared test score: 0.931
0


0.090031713792236373

In [132]:
poly = PolynomialFeatures(degree=1)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)


from sklearn.ensemble import RandomForestRegressor

rfreg = RandomForestRegressor(n_estimators=50, random_state=58).fit(X_train_scaled_poly, y_train)

print('R-squared test score: {:.3f}'
     .format(rfreg.score(X_test_scaled_poly, y_test)))

y_pred2 = rfreg.predict(X_train_scaled_poly)
print(sum(y_pred2<0))

y_pred2[y_pred2<0]=0

y_pred2_rfr = y_pred2

print(rmsle(y_train, y_pred2))

y_pred2_test = rfreg.predict(X_test_scaled_poly)
y_pred2_test_rfr = y_pred2_test
print(sum(y_pred2_test<0))
y_pred2_test[y_pred2_test<0]=0

rmsle(y_test, y_pred2_test)

R-squared test score: 0.944
0
0.0493368708149
0


0.099102888062739036

In [133]:
poly = PolynomialFeatures(degree=1)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

from sklearn.ensemble import GradientBoostingRegressor

gbreg = GradientBoostingRegressor(loss='ls', learning_rate=0.3, n_estimators=100).fit(X_train_scaled_poly, y_train)

print('R-squared test score: {:.3f}'
     .format(gbreg.score(X_test_scaled_poly, y_test)))

print('Feature importances: {}'.format(gbreg.feature_importances_))

y_pred2 = gbreg.predict(X_train_scaled_poly)
print(sum(y_pred2<0))

y_pred2[y_pred2<0]=0

y_pred2_gbr = y_pred2

print(rmsle(y_train, y_pred2))

y_pred2_test = gbreg.predict(X_test_scaled_poly)
y_pred2_test_gbr = y_pred2_test
print(sum(y_pred2_test<0))
y_pred2_test[y_pred2_test<0]=0

rmsle(y_test, y_pred2_test)

R-squared test score: 0.943
Feature importances: [ 0.          0.10181957  0.14149474  0.11665398  0.08147994  0.14002978
  0.17025838  0.10776367  0.14049994]
0
0.0581397386889
0


0.097298523285316005

In [37]:
linreg = Ridge(alpha=0.0001)
linreg_scores = cross_val_score(linreg, X_train_scaled_poly, y_train, cv=2, scoring=rmsle_score)
#linreg_scores = cross_val_score(linreg, X_train_scaled_poly, y_train, cv=3, scoring=mean_squared_log_error)
print(linreg_scores)

[        nan -0.04254513]


/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in log


## Average tests for y2 predicitons

In [128]:
n=6
y_pred2_vote = (1/n)*(y_pred2_ridge + y_pred2_lasso + y_pred2_knn + y_pred2_mlp + y_pred2_rfr + y_pred2_gbr)

print(rmsle(y_train, y_pred2_vote))

n=4
y_pred2_vote = (1/n)*(y_pred2_ridge + y_pred2_mlp + y_pred2_rfr + y_pred2_gbr)

print(rmsle(y_train, y_pred2_vote))

0.0718276938325
0.0648306146232


In [136]:
n=4
y_pred2_test_vote = (1/n)*(y_pred2_test_ridge + y_pred2_test_mlp + y_pred2_test_rfr + y_pred2_test_gbr)



In [169]:
X_train = df[['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]

y_train = df['formation_energy_ev_natom']

X_test = test[['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

linreg = Ridge(alpha=0.0001).fit(X_train_scaled_poly, y_train)


y_pred1 = linreg.predict(X_test_scaled_poly)

In [175]:
X_train = df[['percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]

y_train = df['bandgap_energy_ev']

X_test = test[['percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree']]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
poly = PolynomialFeatures(degree=3)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

linreg = Ridge(alpha=0.0001, max_iter=1000).fit(X_train_scaled_poly, y_train)

y_pred2_ridge = linreg.predict(X_test_scaled_poly)

mlpreg = MLPRegressor(hidden_layer_sizes = [10,10],
                             activation = 'logistic',
                             alpha = 0.003,   #0.0003,
                             solver = 'lbfgs',
                             random_state=101).fit(X_train_scaled_poly, y_train)

y_pred2_mlp = mlpreg.predict(X_test_scaled_poly)

rfreg = RandomForestRegressor(n_estimators=50, random_state=58).fit(X_train_scaled_poly, y_train)

y_pred2_rfr = rfreg.predict(X_test_scaled_poly)

gbreg = GradientBoostingRegressor(loss='ls', learning_rate=0.3, n_estimators=100).fit(X_train_scaled_poly, y_train)

y_pred2_gbr = gbreg.predict(X_test_scaled_poly)

In [176]:
n=4
y_pred2_test_vote = (1/n)*(y_pred2_test_ridge + y_pred2_test_mlp + y_pred2_test_rfr + y_pred2_test_gbr)

print(sum(y_pred1<0))
print(sum(y_pred2_test_vote<0))

y_pred1[y_pred1<0]=0

0
0


In [163]:
submit = {'formation_energy_ev_natom': y_pred1, 'bandgap_energy_ev': y_pred2_test_vote}
submit = pd.DataFrame(data=submit)
cols = submit.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit = submit[cols]
submit.index +=1
submit

,formation_energy_ev_natom,bandgap_energy_ev
1,0.178245,1.936713
2,0.105448,1.723990
3,0.225432,4.246876
4,0.088495,2.997837
5,0.096077,1.244101
6,0.130057,1.523035
7,0.076171,1.484974
8,0.131931,3.164060
9,0.099267,0.429466
10,0.275018,1.244101


In [164]:
submit.to_csv('/Users/jcbrooks/kaggle/nomad_analysis/predictions.csv')

In [178]:
submit = {'formation_energy_ev_natom': y_pred1, 'bandgap_energy_ev': y_pred2_test_vote}
submit = pd.DataFrame(data=submit)
cols = submit.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit = submit[cols]
submit.index +=1
submit

submit.to_csv('/Users/jcbrooks/kaggle/nomad_analysis/predictions.csv')